# Library Installation & Loading

In [2]:
!pip install "../input/segmentation-models-pytorch-021/wheels/pretrainedmodels-0.7.4-py3-none-any.whl"
!pip install "../input/segmentation-models-pytorch-021/wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl"
!pip install "../input/segmentation-models-pytorch-021/wheels/timm-0.4.12-py3-none-any.whl"
!pip install "../input/segmentation-models-pytorch-021/wheels/segmentation_models_pytorch-0.2.1-py3-none-any.whl"
!pip install "../input/monai-wheel/monai-0.8.1-202202162213-py3-none-any.whl"

Processing /kaggle/input/segmentation-models-pytorch-021/wheels/pretrainedmodels-0.7.4-py3-none-any.whl
Processing /kaggle/input/segmentation-models-pytorch-021/wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl
Processing /kaggle/input/segmentation-models-pytorch-021/wheels/timm-0.4.12-py3-none-any.whl
Processing /kaggle/input/segmentation-models-pytorch-021/wheels/segmentation_models_pytorch-0.2.1-py3-none-any.whl
Processing /kaggle/input/monai-wheel/monai-0.8.1-202202162213-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd
import json

# visualization
import cv2
import matplotlib.pyplot as plt
from skimage import io

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold,StratifiedGroupKFold

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torch.nn.functional as F
import segmentation_models_pytorch as smp

import timm

#monai
from monai.metrics.utils import get_mask_edges, get_surface_distance

#keras

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# Configurations

In [4]:
class CFG:
    seed          = 101
    debug         = False # set debug=False for Full Training
    exp_name      = 'v4'
    comment       = 'unet-efficientnet_b0-320x384'
    model_name    = 'Unet'
#     backbone      = 'timm-mobilenetv3_small_minimal_100'
    backbone      = 'efficientnet-b0'
    train_bs      = 32
    valid_bs      = train_bs*2
    img_size      = 320
    epochs        = 1
    lr            = 2e-3
    num_slices    = 5
    in_channels   = 3
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 7
    folds         = [0]
    num_classes   = 3
    thr           = 0.40
#     device        = torch.device("cpu")
    device        = torch.device("cuda")

# Seeding

In [5]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
#     When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     Set a fixed value for the hash seed
    print('> SEEDING DONE')
    
set_seed(CFG.seed)

> SEEDING DONE


# Utilities

In [6]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(mask_img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
#     pixels = img.ravel(order='F')
#     print(type(mask_img))
    pixels = mask_img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    mask_rle = ' '.join(str(x) for x in runs)
    return mask_rle
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = np.tile(img[...,None], [1, 1, 3]) # gray to rgb
    img = img.astype('float32') # original is uint16
    mx = np.max(img)
    if mx:
        img/=mx # scale image to [0, 1]
    return img
def load_slice(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = img.astype('float32') # original is uint16
    #img = cv2.resize(img,(CFG.img_size, CFG.img_size),interpolation = cv2.INTER_AREA)
    return img
def load_5c_img(df, id):
    img = np.zeros((CFG.img_size, CFG.img_size, CFG.in_channels),dtype = np.float32)
    for i, diff in enumerate(range(-(CFG.num_slices//2), CFG.num_slices//2 +1,2)):
        tmp = id.split("_")
        tmp[3] = str(int(tmp[3])+diff).zfill(4)
        id_tmp="_".join(tmp)
        if df["id"].isin([id_tmp]).any().any():
            img[:,:,i] = load_slice(str(df[df["id"]==id_tmp]["image_path"].squeeze()))
        else:
            img[:,:,i] = load_slice(str(df[df["id"]==id]["image_path"].squeeze()))
    max_val = img.max()
    if max_val != 0 :
        img /= max_val
    return img
def load_slice_other(img_file, diff):
    slice_num = os.path.basename(img_file).split('_')[1]
    filename = (
        img_file.replace(
            'slice_' + slice_num,
            'slice_' + str(int(slice_num) + diff).zfill(4)))
    if os.path.exists(filename):
        return cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    return cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
def load_5c_img_other(img_file):
    imgs = [load_slice_other(img_file, i) for i in range(-2, 3,2)]
    img = np.stack(imgs, axis=2)
    img = img.astype(np.float32)
    mx = np.max(img)
    if mx:
        img/=mx # scale image to [0, 1]
    return img

# Metadata

In [7]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
#     row['id'] = f'case{case}_day{day}_slice_{slice_}'
    return row

In [8]:
sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
if not len(sub_df):#during local save version this will activate
    debug = True
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()
else:#during submission this runs
    debug = False
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
sub_df = sub_df.progress_apply(get_metadata,axis=1)

100%|██████████| 1000/1000 [00:01<00:00, 621.79it/s]


In [9]:
if debug:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
#     paths = sorted(paths)
else:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/test/**/*png',recursive=True)
#     paths = sorted(paths)
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.progress_apply(path2info, axis=1)
path_df.head()

100%|██████████| 38496/38496 [01:30<00:00, 427.54it/s]


,image_path,height,width,case,day,slice
0,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,6
1,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,82
2,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,113
3,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,76
4,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,125


In [10]:
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')
test_df.head()

,id,case,day,slice,image_path,height,width
0,case123_day20_slice_0001,123,20,1,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
1,case123_day20_slice_0002,123,20,2,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
2,case123_day20_slice_0003,123,20,3,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
3,case123_day20_slice_0004,123,20,4,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266
4,case123_day20_slice_0005,123,20,5,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266


# Mask creation + dataset creation

In [11]:
def id2mask(id_):
    idf = df[df['id']==id_]
    shape = (df[df['id']==id_]["height"].squeeze(), idf["width"].squeeze(), 3)
    mask = np.zeros(shape, dtype=np.uint8)
    for i, rle in enumerate(idf["segmentation"].squeeze()):
        if pd.notnull(rle):
            mask[:,:, i] = rle_decode(rle, shape[:2])
    return mask

# print(id2mask("case123_day20_slice_0066"))

In [12]:
data_transforms = {
        "train": A.Compose([
                    A.Resize(CFG.img_size,CFG.img_size, interpolation=cv2.INTER_NEAREST),
            A.HorizontalFlip(p=0.5),
#             A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
            A.OneOf([
                A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
                A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
            ], p=0.25),
            A.CoarseDropout(max_holes=8, max_height=CFG.img_size // 20, max_width=CFG.img_size // 20,
                            min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        ], p=1.0),

        "valid": A.Compose([A.Resize(CFG.img_size, CFG.img_size, interpolation=cv2.INTER_NEAREST)], p=1.0)
    }

In [13]:
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=False, transforms=None):
        self.df         = df
        self.label      = label
        self.img_paths  = df['image_path'].tolist()
        self.ids        = df['id'].tolist()
        if 'msk_path' in df.columns:
            self.msk_paths  = df['mask_path'].tolist()
        else:
            self.msk_paths = None
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        id_       = self.ids[index]
        img = []
#         img = load_5c_img(self.df, id_)
        img = load_5c_img_other(img_path)
        h, w = img.shape[:2]
        if self.label:
            msk_path = self.msk_paths[index]
            msk = load_msk(msk_path)
            if self.transforms:
                data = self.transforms(image=img, mask=msk)
                img  = data['image']
                msk  = data['mask']
            img = np.transpose(img, (2, 0, 1))
            msk = np.transpose(msk, (2, 0, 1))
            return torch.tensor(img), torch.tensor(msk)
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img  = data['image']
            img = np.transpose(img, (2, 0, 1))
            return torch.tensor(img), id_, h, w

In [14]:

def build_model():
    model = smp.UnetPlusPlus(
        encoder_name=CFG.backbone,      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#         encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        encoder_weights= None,
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
        decoder_attention_type='scse',
    )
    model.to(CFG.device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [15]:
# import cupy as cp

def mask2rle(msk, thr=0.5):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    msk    = np.array(msk)
    pixels = msk.flatten()
    pad    = np.array([0])
    pixels = np.concatenate([pad, pixels, pad])
    runs   = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def masks2rles(msks, ids, heights, widths):
    pred_strings = []; pred_ids = []; pred_classes = [];
    for idx in range(msks.shape[0]):
        height = heights[idx].item()
        width = widths[idx].item()
        msk = cv2.resize(msks[idx], 
                         dsize=(width, height), 
                         interpolation=cv2.INTER_NEAREST) # back to original shape
        rle = [None]*3
        for midx in [0, 1, 2]:
            rle[midx] = mask2rle(msk[...,midx])
        pred_strings.extend(rle)
        pred_ids.extend([ids[idx]]*len(rle))
        pred_classes.extend(['large_bowel', 'small_bowel', 'stomach'])
    return pred_strings, pred_ids, pred_classes

In [16]:
@torch.no_grad()
def infer(model_path, test_loader, num_log=1, thr=CFG.thr):
    msks = []; imgs = [];
    pred_strings = []; pred_ids = []; pred_classes = [];
    for idx, (img, ids, heights, widths) in enumerate(tqdm(test_loader, total=len(test_loader), desc='Infer ')):
        img = img.to(CFG.device, dtype=torch.float) # .squeeze(0)
        size = img.size()
        msk = []
        msk = torch.zeros((size[0], 3, size[2], size[3]), device=CFG.device, dtype=torch.float32)
        
        model = load_model(model_path)
        out   = model(img) # .squeeze(0) # removing batch axis
        out   = nn.Sigmoid()(out) # removing channel axis
        msk+=out
        
        msk = (msk.permute((0,2,3,1))>thr).to(torch.uint8).cpu().detach().numpy() # shape: (n, h, w, c)
        result = masks2rles(msk, ids, heights, widths)
        pred_strings.extend(result[0])
        pred_ids.extend(result[1])
        pred_classes.extend(result[2])
        if idx<num_log:
            img = img.permute((0,2,3,1)).cpu().detach().numpy()
            imgs.append(img[:10])
            msks.append(msk[:10])
        del img, msk, out, model, result
        torch.cuda.empty_cache()
    return pred_strings, pred_ids, pred_classes, imgs, msks

In [17]:
test_dataset = BuildDataset(test_df, transforms=data_transforms['valid'])
test_loader  = DataLoader(test_dataset, batch_size=CFG.valid_bs, 
                          num_workers=4, shuffle=False, pin_memory=False)
model_path="../input/uwmgi/last_epoch-00.bin"
pred_strings, pred_ids, pred_classes, imgs, msks = infer(model_path, test_loader)

Infer : 100%|██████████| 16/16 [00:36<00:00,  2.29s/it]


In [18]:
pred_df = pd.DataFrame({
    "id":pred_ids,
    "class":pred_classes,
    "predicted":pred_strings
})
if not debug:
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    del sub_df['predicted']
else:
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]
    del sub_df['segmentation']
    
sub_df = sub_df.merge(pred_df, on=['id','class'])
sub_df.to_csv('submission.csv',index=False)
display(sub_df.head(5))

,id,class,predicted
0,case123_day20_slice_0001,large_bowel,
1,case123_day20_slice_0001,small_bowel,
2,case123_day20_slice_0001,stomach,
3,case123_day20_slice_0002,large_bowel,
4,case123_day20_slice_0002,small_bowel,
